In [1]:
import pandas as pd
from GPTModel import GPTInference
import spacy
import re, string
nlp = spacy.load("en_core_web_sm")

2023-10-13 13:09:48.829732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 13:09:49.616536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_aggrefact.csv')

df.head()

,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,CircE_text,...,GramE_text,PredE,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text,origin
0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval
4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,FacEval


In [3]:
df = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/llm_annotations/sample/sample_annotations_sanjana.csv')

df.head()

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
0,16,3cc5013c-4080-413e-aa5f-aec0dc93e349,sanjana,13681870,gpt4-32k-0613,NaN,NaN,NaN,Miley expresses her fatigue and lack of desire...,Miley: don''t want to go to work tomorrow!\r\n...
1,27,c094b128-08b6-4187-91cf-c44d9ab25baa,sanjana,13681871,gpt4-32k-0613,NaN,NaN,NaN,Andy informs Sue that he will be working late ...,Andy: Working late til 4 xx\r\nSue: no worries...
2,32,149452c6-bf79-47b0-b0ee-ecfcf99caa7f,sanjana,13681429,gpt4-32k-0613,NaN,NaN,NaN,Cass confirmed with Jordan that he received hi...,"Cass: Hi darling, did you get your birthday ca..."
3,34,a8c98d99-c964-46d0-a51c-bca6569900c1,sanjana,13681439,gpt4-32k-0613,NaN,NaN,NaN,Phoebe can''t go out because her mother is ang...,Phil: can you go out today?\r\nPhoebe: no\r\nP...
4,36,48d59334-9492-4cc1-a5ce-1b0378e34a1f,sanjana,13681441,gpt4-32k-0613,NaN,NaN,NaN,Joanna shares with Merve that their mutual acq...,Joanna: They are sending emails about Lewandow...


In [3]:
df = df[df['origin'] != 'RefMatters_DialogSu']

In [4]:

def get_atomic_facts_gpt(gpt_model_atomic, text, text_type):
    instr = f'Segment the following {text_type} into important facts without introducing any unsupported information'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt)
    return gpt_response

def mask_all_errtypes(text):
    doc = nlp(text)
    origin_tokens = [token for token in doc]
    all_masked_results = []
    mask_idx = []
    mask_tokens = []
    for idx, token in enumerate(doc):
        append = False
        if 'subj' in token.dep_ or 'obj' in token.dep_:
            # print(idx, origin_tokens)
            # masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            # masked_text = ' '.join(masked_text)
            # print(masked_text)
            append = True
            token_type = 'subjobj'
            
        elif 'VERB' == token.pos_:
            append = True 
            token_type = 'predicate'

        elif 'ADV' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'ADP' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'PRON' == token.pos_:
            append = True
            token_type = 'coreference'

        

        if append:
            mask_idx.append(idx)
            mask_tokens.append((idx, token.text, token_type))
    # print([])
    # print(mask_tokens)  

    error_types_indices_map = {}
    for token_idx, token_text, err_type in mask_tokens:
        if err_type not in error_types_indices_map:
            error_types_indices_map[err_type] = []
        error_types_indices_map[err_type] += [token_idx]
        
    for err_type, err_indices in error_types_indices_map.items():
        for idx in err_indices:
            masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            masked_text = ' '.join(masked_text)
            all_masked_results.append((masked_text, origin_tokens[idx].text, err_type))
    return all_masked_results
    

In [5]:
def get_predicate_answer(gpt_model_corruptor, dlg, atomic_facts, summary, answer):
    instruction_verb = f'List alternative words or phrases to "{answer}" can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue.'
    instruction_tense = f'List different tenses of verbs or phrases of the word {answer} that can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
    instruction_negation = f"List words or phrases that negate {answer} and can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts."
    answer_all = []
    
    prompt_template_verb = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction_verb}\nAnswer:'
    answer_verb = gpt_model_corruptor.get_chatgpt_response(prompt_template_verb)
    # print('ANSWER VERB', answer_verb)
    if len(answer_verb.split(',')) == 1:
        answer_all += answer_verb.split('\n')
    else:
        answer_all += answer_verb.split(',')
    
    
    prompt_template_tense  = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction_tense}\nAnswer:'
    answer_tense = gpt_model_corruptor.get_chatgpt_response(prompt_template_tense)
    # print('ANSWER TENSE', answer_tense)
    if len(answer_tense.split(',')) == 1:
        answer_all += answer_tense.split('\n')
    else:
        answer_all += answer_tense.split(',')
    
    prompt_template_negation = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction_negation}\nAnswer:'
    answer_negation = gpt_model_corruptor.get_chatgpt_response(prompt_template_negation)
    # print('ANSWER NEGATION', answer_negation)
    if len(answer_negation.split(',')) == 1:
        answer_all += answer_negation.split('\n')
    else:
        answer_all += answer_negation.split(',')
        
    answer_all = ','.join(answer_all)
    return answer_all


def get_generic_answer(gpt_model_corruptor, dlg, atomic_facts, summary, answer):
    instruction = f'List alternative words or phrases to "{answer}" can be used to fill in the <BLANK> of the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
    prompt_template = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction}\nAnswer:'
    answer = gpt_model_corruptor.get_chatgpt_response(prompt_template)
    if len(answer.split(',')) == 1:
        answer = ','.join(answer.split('\n'))
    
    return answer
    
def check_sentence_validity(gpt_model_sentence_checker, text):
    instruction = "Correct words in the sentence that are grammatically  wrong for the sentence"
    prompt_template = f'{instruction}\nSummary: {text}\nCorrected Summary:'
    answer = gpt_model_sentence_checker.get_chatgpt_response(prompt_template)
    return answer

def check_entailment(gpt_model_nli, dlg, atomic_facts, text):
    instruction = "Does the dialogue or facts entail the sentence?"
    prompt_template = f'{instruction}\nDialogue: {dlg}\nFacts: {atomic_facts}\nSentence: {text}\nAnswer(yes or no):'
    answer = gpt_model_nli.get_chatgpt_response(prompt_template)
    return answer

In [6]:
import string 
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english')


def make_corrupted_summary(gpt_model_corruptor, dlg, atomic_fact_blank, answer, err_type, facts):
    corrupted_err_type = []
    # for blanked_text, answer, err_type in text_blanks:
        # if err_type == 'subjobj':
        #     corrupted_answer = get_generic_answer(gpt_model_corruptor, dlg, facts, blanked_text, answer)
        #     # print(blanked_text, err_type)
        #     # print(answer)
        #     # print(corrupted_answer)
        #     # print('====')
    if err_type == 'predicate':
            corrupted_answer = get_predicate_answer(gpt_model_corruptor, dlg, facts, atomic_fact_blank, answer)
            # print(atomic_fact_blank, err_type)
            # print(answer)
            # print(corrupted_answer)
            # print('====')
        
    else:
        corrupted_answer = get_generic_answer(gpt_model_corruptor, dlg, facts, atomic_fact_blank, answer)
    
        
    options = corrupted_answer.split(',') 
    # if len(options) == 1:
    #     options = corrupted_answer.split('\n')
    options = sample(options, 8) if len(options) >= 8 else options
    options = [re.sub('[1-9]', '', each) for each in options]
    # print('OPTIONS', options)
    corrupted_facts = []
    for option in options:
            corrupted_summary = atomic_fact_blank.replace('<BLANK>', option.strip())
            corrected_summary = check_sentence_validity(gpt_model_sentence_checker, corrupted_summary)
            
            corrected_summary = [each.strip(string.punctuation) for each in corrected_summary.split(' ') ]
            # corrected_summary = [each for each in corrected_summary if not each.isdigit()]
            corrupted_summary = [each.strip(string.punctuation) for each in corrupted_summary.split(' ')]
            # corrupted_summary = [each for each in corrupted_summary if not each.isdigit()]
            
            # word_constraints = [each for each in atomic_fact_blank.split(' ') if each != '<BLANK>' and each not in stop_words]
            # constrain_overlap = set(word_constraints) - set(corrected_summary)
            replaced_word = ' '.join([each for each in corrected_summary if each.strip(string.punctuation).lower() not in atomic_fact_blank.lower()])
    
            corrected_summary = ' '.join(corrected_summary)
            corrupted_summary = ' '.join(corrupted_summary)
            # print(corrected_summary)
            # if replaced_word: 
            entailment = check_entailment(gpt_model_nli, dlg, facts, corrected_summary)
                
            # print(corrected_summary, entailment)
            # print(replaced_word)
            # print('----')
            if entailment == 'no' and replaced_word in corrected_summary:
                    # if err_type not in corrupted_err_type:
                    #     corrupted_err_type[err_type] = []
                    corrupted_err_type += [(corrected_summary, replaced_word)]
    return corrupted_err_type

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sanjana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
gpt_model_corruptor = GPTInference()
gpt_model_sentence_checker = GPTInference()
gpt_model_atomic = GPTInference()
gpt_model_nli = GPTInference()

In [8]:
from random import sample
import nltk
def get_atomic_facts_gpt(gpt_model_atomic, model, text, text_type):
    instr = f'Write the {text_type} into indirect speech without introducing any unsupported information or inferences'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt, model = model)
    # print(nltk.sent_tokenize(gpt_response))
    return gpt_response
    
class SyntheticPrompt:

    def __init__(self):
        self.gpt_model_corruptor = GPTInference()
        self.gpt_model_sentence_checker = GPTInference()
        self.gpt_model_atomic = GPTInference()
        self.gpt_model_nli = GPTInference()

    def get_atomic_facts(self, gpt_model_type, dlg):
        atomic_facts = get_atomic_facts_gpt(self.gpt_model_atomic, gpt_model_type, dlg, 'dialogue')
        atomic_facts = nltk.sent_tokenize(atomic_facts)
        atomic_facts = [re.sub('[1-9]', '', each) for each in atomic_facts]
        atomic_facts = [each.strip(string.punctuation).strip() for each in atomic_facts]
        return atomic_facts

    def make_masked_sentences(self, atomic_facts):
        masked_atomic_facts_map = {}
        for atomic_fact in atomic_facts:
            masked_results = mask_all_errtypes(atomic_fact)
            for sent, ans, err_type in masked_results:
                if err_type not in masked_atomic_facts_map:
                    masked_atomic_facts_map[err_type] = []
                masked_atomic_facts_map[err_type].append((atomic_fact, sent, ans, err_type))

        return masked_atomic_facts_map

    def get_corrupted_sent_spans(self, dlg, atomic_facts, sample_facts):
        atomic_fact_map = {}
        facts = '\n'.join(atomic_facts)
        # print('SAMPLE', sample_facts)
        for atomic_fact, atomic_fact_blank, answer, err_type in sample_facts:
            print(err_type)
            atomic_fact_corrupted = make_corrupted_summary(self.gpt_model_corruptor, 
                                                               dlg, 
                                                               atomic_fact_blank, 
                                                               answer, 
                                                               err_type, 
                                                               facts)
            # print(atomic_fact_corrupted)
            # print('======')
            if atomic_fact not in atomic_fact_map:
                atomic_fact_map[atomic_fact] = {}
                
            for corrupted_summary, replaced_span in atomic_fact_corrupted:
                if err_type not in atomic_fact_map[atomic_fact]:
                    atomic_fact_map[atomic_fact][err_type] = []
                atomic_fact_map[atomic_fact][err_type] += [(corrupted_summary, replaced_span)]
        return atomic_fact_map

    def get_single_err_samples(self, num_single_err_samples, atomic_fact_map, facts_list):
        corrupted_summ_sent_spans = []
        all_summary_sents = list(atomic_fact_map.keys())
        num_single_err_samples = num_single_err_samples if len(all_summary_sents) >= num_single_err_samples else len(all_summary_sents)
        picked_sents = sample(all_summary_sents, num_single_err_samples)
        shortlisted_err_types = []
        for picked_sent in picked_sents:
            picked_sent_idx = facts_list.index(picked_sent)
            for k , v in atomic_fact_map[picked_sent].items():
                if k not in shortlisted_err_types:
                    corrupted_pick_sent_span = sample(v, 1)[0]
                    corrupted_summary = facts_list[:picked_sent_idx] + [corrupted_pick_sent_span[0]] + facts_list[picked_sent_idx + 1:]
                    corrupted_summ_sent_spans.append(('. '.join(corrupted_summary), 
                                                  [(corrupted_pick_sent_span[0], 
                                                  corrupted_pick_sent_span[1])]))
                    shortlisted_err_types.append(k)
        return corrupted_summ_sent_spans

    def make_single_error_samples(self, dlg, summary, num_samples = 4):
        atomic_facts_models = ['gpt-4-32k-0613', 'gpt-3.5-turbo-16k-0613', 'gpt-4-32k-0314']
        corrupted_summ_sent_spans = []
        for gpt_model_type in atomic_facts_models[:1]:
            atomic_facts = self.get_atomic_facts(gpt_model_type, dlg)
            print(gpt_model_type, atomic_facts)
            masked_atomic_facts_map = self.make_masked_sentences(atomic_facts)
            # print(masked_atomic_facts_map)
            sample_facts = []
            for k , v in masked_atomic_facts_map.items():
                num_samples = 2 if len(v) >=2 else len(v)
                sample_facts += sample(v, num_samples)

            print('SAMPLE', sample_facts)
            print('***'*13)
            atomic_fact_map = self.get_corrupted_sent_spans(dlg, atomic_facts, sample_facts)
            
            instruction = "Append 1-2 words or a phrase to the sentence with extra information"
            for atomic_fact in sample(atomic_facts, 2):
                            # atomic_fact = atomic_fact.strip(string.punctuation)
                            prompt_template = f'Sentence: {atomic_fact}\n{instruction}\n'
                            gpt_summ = self.gpt_model_corruptor.get_chatgpt_response(prompt_template)
                            gpt_summ = gpt_summ.strip(string.punctuation)
                            picked_sent_idx = atomic_facts.index(atomic_fact)
                            corrupted_summary = atomic_facts[:picked_sent_idx] + [gpt_summ] + atomic_facts[picked_sent_idx + 1:]
                            atomic_fact = atomic_fact.strip(string.punctuation)
                            added_span = ' '.join([each for each in gpt_summ.split(' ') if each.strip(string.punctuation).lower() not in atomic_fact.lower()])
                            corrupted_summ_sent_spans.append((('. '.join(corrupted_summary)),
                                                                     [(gpt_summ, added_span)]))
                            print("OOE", (('. '.join(corrupted_summary)),
                                                                     [(gpt_summ, added_span)]))
            corrupted_summ_sent_spans += self.get_single_err_samples(5, atomic_fact_map, atomic_facts)
        
        return corrupted_summ_sent_spans
            
        

In [9]:
# row = df[~df['nonfactual_spans'].isnull()].sample(1)
row = df.iloc[[166]]
print(row['dialogue'].values[0])
print(row['summary'].values[0])
print(row['nonfactual_spans'].values[0])
text = row['summary'].values[0]
dlg = row['dialogue'].values[0]

Julie: hey guys... could you just talk to me for a bit? I just watched this Japanese horror movie and I''m home alone and a little uneasy (aka scared shitless)
Rose: Jesus, why on earth would you watch a Japanese horror, home alone at this hour?
Julie: Cause I''m a fucking moron?
Rose: Cause you''re a fuckin moron. 
Paula: 5 point for Gryffindor, my friend. But in your defense, I always thought you were quite unaffected by horror movies.
Julie: That''s what I thought too!
Paula: So what kinda movie was it?
Julie: <file_other>
Rose: seems pretty generic
Julie: It''s scarier than it looks.
Rose: you know that japanese horror are the worst ones, right?
Julie: I know that now.
Paula: If... you want me to come over I can.
Julie: omg really? <3
Paula: Yeah, sure, it''s like 20 minutes by bike to your house, we can drink cocoa and watch "when harry met Sally" until we fall asleep ;)
Julie: Omg Thank you so much, I love you!
Rose: I wish I lived in your neighborhood :(
Paula: We can chip in fo

In [13]:
df = df[~df['nonfactual_spans'].isnull()]
len(df)

32

In [320]:
text = row['summary'].values[0]
dlg = row['dialogue'].values[0]
synthetic_generator = SyntheticPrompt()
corrupted_summ_sent_spans = synthetic_generator.make_single_error_samples(dlg, text)

Write the dialogue into indirect speech without introducing any unsupported information or inferences
Dialogue: Eric: MACHINE!
Rob: That''s so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it''s really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I''ll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I''ll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
gpt-4-32k-0613 ['Eric called out "MACHINE', 'and Rob agreed enthusiastically that it was great', 'Eric acknowledged this agreement by explaining that it reflects how Americans perceive Russians', 'Rob echoed this sentiment, finding the whole thing quite amusing', 'Eric agreed, and expressed a particular fondness for the part with the train', 'Rob laughed and commented that no one communicates with the machine the way it was shown', 'Er

In [337]:
ic_examples = []
for summ_spans in corrupted_summ_sent_spans:
    list_errors = '\n\n'.join([f' { each[1]}' for each in summ_spans[1]])
    ic_ex = f'Summary: {summ_spans[1][0][0],}\nInconsistent sentence and span: {list_errors}'
    # print(ic_ex)
    ic_examples += [ic_ex]

In [338]:
ic_examples

["Summary: ('Rob echoed this sentiment, finding the whole thing quite amusing despite the seriousness of the situation',)\nInconsistent sentence and span:  despite seriousness of situation",
 'Summary: ("Rob admitted he wasn\'t sure but promised to check his work schedule",)\nInconsistent sentence and span:  his work schedule',
 "Summary: ('The machine agreed and expressed a particular fondness for the part with the train',)\nInconsistent sentence and span:  machine",
 "Summary: ('Eric was excited to hear this and said he would watch them annually',)\nInconsistent sentence and span:  annually",
 "Summary: ('He later confirmed that more dance-up performances by the same performer were available on YouTube',)\nInconsistent sentence and span:  dance-up",
 "Summary: ('He later confirmed that no other stand-up performances by the same performer were available on YouTube',)\nInconsistent sentence and span:  no other stand-up"]

In [339]:
inconsistent_instruction = 'Find all spans in the summary that is inconsistent with the source. Note that consistency means all information in the summary is supported by the source.' 
ic_examples = '\n'.join(ic_examples)
# atomic_facts = self.get_atomic_facts('', dlg)
atomic_facts = SyntheticPrompt().get_atomic_facts('gpt-4-32k-0613', dlg)
atomic_facts = '. '.join(atomic_facts)
for txt in nltk.sent_tokenize(text):
    prompt_template = f'Source: {atomic_facts}\n{inconsistent_instruction}\n{ic_examples}\nSummary: {txt}\nInconsistent sentence/spans:'

    # print(prompt_template)
    print(txt)
    print('ANSWER', GPTInference().get_chatgpt_response(prompt_template, model = "gpt-4-32k-0613"))
    

Write the dialogue into indirect speech without introducing any unsupported information or inferences
Dialogue: Eric: MACHINE!
Rob: That''s so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it''s really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I''ll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I''ll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
Eric and Rob are discussing a stand-up comedy routine they both enjoyed, particularly a segment involving a train and a character referred to as MACHINE.
ANSWER None
They find humor in the exaggerated portrayal of a Russian character and decide to look up more of the comedian''s work on Youtube.
ANSWER None
They affirm their enjoyment by repeating the phrase "MACHINE!"
ANSWER They affirm their enjoyment by repeating the phrase "MACHINE

In [442]:
import nltk 
inconsistent_instruction = 'Find spans in the summary that is inconsistent with the source. Note that consistency means all information in the summary is supported by the source.' 
# ic_examples = '\n'.join(ic_examples)
# atomic_facts = self.get_atomic_facts('', dlg)
text = row['summary'].values[0]
dlg = row['dialogue'].values[0]
atomic_facts = SyntheticPrompt().get_atomic_facts('gpt-4-32k-0613', dlg)
atomic_facts = '. '.join(atomic_facts)
for txt in nltk.sent_tokenize(text):
    prompt_template = f'Source: {atomic_facts}\n{inconsistent_instruction}\nSummary: {txt}\nInconsistent sentence/spans:'

    print(txt)
    # print(prompt_template)
    print('ANSWER', GPTInference().get_chatgpt_response(prompt_template, model = "gpt-4-32k-0613"))
    print('==' * 13)

Write the dialogue into indirect speech without introducing any unsupported information or inferences
Dialogue: Julie: hey guys... could you just talk to me for a bit? I just watched this Japanese horror movie and I''m home alone and a little uneasy (aka scared shitless)
Rose: Jesus, why on earth would you watch a Japanese horror, home alone at this hour?
Julie: Cause I''m a fucking moron?
Rose: Cause you''re a fuckin moron. 
Paula: 5 point for Gryffindor, my friend. But in your defense, I always thought you were quite unaffected by horror movies.
Julie: That''s what I thought too!
Paula: So what kinda movie was it?
Julie: <file_other>
Rose: seems pretty generic
Julie: It''s scarier than it looks.
Rose: you know that japanese horror are the worst ones, right?
Julie: I know that now.
Paula: If... you want me to come over I can.
Julie: omg really? <3
Paula: Yeah, sure, it''s like 20 minutes by bike to your house, we can drink cocoa and watch "when harry met Sally" until we fall asleep ;)

In [375]:
print(prompt)

NameError: name 'prompt' is not defined

In [353]:
masked_sent = "They find humor in the exaggerated portrayal of a Russian <BLANK> and decide to look up more of the comedian''s work on Youtube."
# List top 10 words to fill in the <BLANK>


In [352]:
atomic_facts

'Eric excitedly mentioned something called the "machine", to which Rob agreed was great. Eric then expressed his views that the machine shows how Americans perceive Russians, to which Rob added that he found the concept hilariously funny. Eric agreed again and added that he particularly enjoyed the part concerning the train. They both then laughed about how no one talks to the machine like that. Eric inquired if this machine was part of a stand-up, and Rob admitted his ignorance and promised to check. After checking, Rob informed Eric that there were indeed additional stand-ups available on YouTube. Thrilled, Eric declared he\'d watch them right away, a sentiment Rob shared. They finished the conversation by repeating the word "machine" and agreed to talk at a later time'

In [366]:
import os
import openai
import numpy as np
def logprob_to_prob(logprob: float) -> float:
    return np.exp(logprob)
    
# openai.api_key = os.getenv("OPENAI_API_KEY")
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=f"Source:{dlg}Summary:{masked_sent}Complete the summary by filling in the blanks",
  max_tokens=100,
  temperature=0,
    logprobs = 5
)

In [368]:
masked_sent

"They find humor in the exaggerated portrayal of a Russian <BLANK> and decide to look up more of the comedian''s work on Youtube."

In [367]:
for k , v in response['choices'][0]['logprobs']['top_logprobs'][1].items():
    print(k, logprob_to_prob(v))

inter 0.3657226206440408
person 0.18828358911214307
man 0.17113341876537475
con 0.05134731151828027
character 0.046407857101559105


In [443]:
inconsistent_instruction = 'Find spans in the summary that is inconsistent with the source. Note that consistency means all information in the summary is supported by the source.'
ic_examples = '\n'.join(ic_examples)

prompt_template = f'Source: {atomic_facts}\n{inconsistent_instruction}\nSummary: {text}\nInconsistent sentence/spans:'

print(prompt_template)
GPTInference().get_chatgpt_response(prompt_template)

Source: Julie had requested her friends to chat with her as she had just finished watching a Japanese horror movie and was home alone, feeling very frightened. Rose wondered why Julie would choose to watch such a movie so late at night while being by herself. Julie confessed she had been foolish to do so, and Rose agreed. Paula made a light-hearted reference to a popular fantasy series and defended Julie by stating she believed Julie was generally unphased by horror films. Julie admitted she had thought the same. In their conversation, Paula asked Julie about the kind of movie she had watched. Julie responded, and while Rose commented that it seemed rather standard, Julie maintained it was more terrifying than it appeared. Rose reminded Julie of the often intense nature of Japanese horror films, a fact Julie confessed she only now realized. Paula offered to cycle over to Julie's house, suggesting they drink cocoa and watch a more lighthearted movie until they fall asleep. Overjoyed, Ju

'"Rose criticizes Julie for watching a horror movie alone, despite being aware of their intensity."'

In [53]:
GPTInference().get_chatgpt_response(prompt_template)

'"on their success tonight"'

In [177]:
masked_atomic_facts_map = {}
for atomic_fact in texts:
    masked_results = mask_all_errtypes(atomic_fact)
    for sent, ans, err_type in masked_results:
        if err_type not in masked_atomic_facts_map:
            masked_atomic_facts_map[err_type] = []
        masked_atomic_facts_map[err_type].append((atomic_fact, sent, ans, err_type))


[(0, 'Greg', 'subjobj'), (1, 'needs', 'predicate'), (3, 'stay', 'predicate'), (4, 'after', 'circumstance'), (5, 'hours', 'subjobj'), (6, 'at', 'circumstance'), (7, 'work', 'subjobj')]
[(0, 'Betsy', 'subjobj'), (1, 'expresses', 'predicate'), (2, 'frustration', 'subjobj'), (3, 'at', 'circumstance'), (5, 'repeated', 'predicate'), (6, 'situation', 'subjobj')]
[(0, 'Johnny', 'subjobj'), (1, 'needs', 'predicate'), (4, 'picked', 'predicate'), (5, 'up', 'circumstance'), (6, 'from', 'circumstance'), (7, 'kindergarten', 'subjobj')]
[(0, 'Greg', 'subjobj'), (1, 'asks', 'predicate'), (2, 'Betsy', 'subjobj'), (4, 'she', 'subjobj'), (6, 'pick', 'predicate'), (7, 'up', 'circumstance'), (8, 'Johnny', 'subjobj')]
[(0, 'Betsy', 'subjobj'), (3, 'pick', 'predicate'), (4, 'up', 'circumstance'), (5, 'Johnny', 'subjobj'), (7, 'she', 'subjobj'), (8, 'also', 'circumstance'), (9, 'needs', 'predicate'), (11, 'work', 'predicate')]
[(0, 'Tuesdays', 'subjobj'), (2, 'designated', 'predicate'), (3, 'as', 'circumstanc

In [178]:
masked_atomic_facts_map

{'subjobj': [('Greg needs to stay after hours at work',
   '<BLANK> needs to stay after hours at work',
   'Greg',
   'subjobj'),
  ('Greg needs to stay after hours at work',
   'Greg needs to stay after <BLANK> at work',
   'hours',
   'subjobj'),
  ('Greg needs to stay after hours at work',
   'Greg needs to stay after hours at <BLANK>',
   'work',
   'subjobj'),
  ('Betsy expresses frustration at the repeated situation',
   '<BLANK> expresses frustration at the repeated situation',
   'Betsy',
   'subjobj'),
  ('Betsy expresses frustration at the repeated situation',
   'Betsy expresses <BLANK> at the repeated situation',
   'frustration',
   'subjobj'),
  ('Betsy expresses frustration at the repeated situation',
   'Betsy expresses frustration at the repeated <BLANK>',
   'situation',
   'subjobj'),
  ('Johnny needs to be picked up from kindergarten',
   '<BLANK> needs to be picked up from kindergarten',
   'Johnny',
   'subjobj'),
  ('Johnny needs to be picked up from kindergarten

In [179]:
from random import sample
sample_facts = []
for k , v in masked_atomic_facts_map.items():
    num_samples = 3 if len(v) >=3 else len(v)
    sample_facts += sample(v, num_samples)

In [180]:
sample_facts

[('Betsy cannot pick up Johnny because she also needs to work long hours',
  'Betsy can not pick up <BLANK> because she also needs to work long hours',
  'Johnny',
  'subjobj'),
 ('Betsy cannot pick up Johnny because she also needs to work long hours',
  'Betsy can not pick up Johnny because <BLANK> also needs to work long hours',
  'she',
  'subjobj'),
 ("Tuesdays are designated as Greg's days to handle kindergarten pick-up",
  "<BLANK> are designated as Greg 's days to handle kindergarten pick - up",
  'Tuesdays',
  'subjobj'),
 ('Johnny needs to be picked up from kindergarten',
  'Johnny needs to be <BLANK> up from kindergarten',
  'picked',
  'predicate'),
 ('Greg agrees to find a solution for picking up Johnny',
  'Greg agrees to find a solution for <BLANK> up Johnny',
  'picking',
  'predicate'),
 ("Tuesdays are designated as Greg's days to handle kindergarten pick-up",
  "Tuesdays are <BLANK> as Greg 's days to handle kindergarten pick - up",
  'designated',
  'predicate'),
 ('G

In [181]:
atomic_fact_map = {}
facts = '\n'.join(texts)
for atomic_fact, atomic_fact_blank, answer, err_type in sample_facts:
# for atomic_fact in texts:
#     atomic_fact_blanks = mask_all_errtypes(atomic_fact)
    atomic_fact_corrupted = make_corrupted_summary(gpt_model_corruptor, dlg, atomic_fact_blank, answer, err_type, facts)
    print(err_type, atomic_fact_corrupted)
    print('*****')
    if atomic_fact not in atomic_fact_map:
        atomic_fact_map[atomic_fact] = {}
    
    for corrupted_summary, replaced_span in atomic_fact_corrupted:
        if err_type not in atomic_fact_map[atomic_fact]:
            atomic_fact_map[atomic_fact][err_type] = []
        atomic_fact_map[atomic_fact][err_type] += [(corrupted_summary, replaced_span)]
    # atomic_fact_map[atomic_fact] += atomic_fact_corrupted

OPTIONS ['1. groceries', '2. her sister', '3. the dry cleaning', '4. their dog', '5. dinner']
Betsy can not pick up <BLANK> because she also needs to work long hours
Betsy cannot pick up groceries because she also needs to work long hours no
cannot groceries
----
Betsy can not pick up <BLANK> because she also needs to work long hours
Betsy cannot pick up her sister because she also needs to work long hours no
cannot her sister
----
Betsy can not pick up <BLANK> because she also needs to work long hours
Betsy cannot pick up the dry cleaning because she also needs to work long hours no
cannot the dry cleaning
----
Betsy can not pick up <BLANK> because she also needs to work long hours
Betsy cannot pick up their dog because she also needs to work long hours no
cannot their dog
----
Betsy can not pick up <BLANK> because she also needs to work long hours
Betsy cannot pick up dinner because she also needs to work long hours no
cannot dinner
----
subjobj [('Betsy cannot pick up groceries beca

In [182]:
atomic_fact_map

{'Betsy cannot pick up Johnny because she also needs to work long hours': {'subjobj': [('Betsy cannot pick up groceries because she also needs to work long hours',
    'cannot groceries'),
   ('Betsy cannot pick up her sister because she also needs to work long hours',
    'cannot her sister'),
   ('Betsy cannot pick up the dry cleaning because she also needs to work long hours',
    'cannot the dry cleaning'),
   ('Betsy cannot pick up their dog because she also needs to work long hours',
    'cannot their dog'),
   ('Betsy cannot pick up dinner because she also needs to work long hours',
    'cannot dinner'),
   ('Betsy cannot pick up Johnny because Greg also needs to work long hours',
    'cannot Greg'),
   ('Betsy cannot pick up Johnny because Johnny also needs to work long hours',
    'cannot'),
   ('Betsy cannot pick up Johnny because their neighbor also needs to work long hours',
    'cannot their neighbor'),
   ('Betsy cannot pick up Johnny because the babysitter also needs to 

In [183]:
texts

['Greg needs to stay after hours at work',
 'Betsy expresses frustration at the repeated situation',
 'Johnny needs to be picked up from kindergarten',
 'Greg asks Betsy if she can pick up Johnny',
 'Betsy cannot pick up Johnny because she also needs to work long hours',
 "Tuesdays are designated as Greg's days to handle kindergarten pick-up",
 'Greg agrees to find a solution for picking up Johnny']

In [186]:
num_single_err_samples = 1
corrupted_summ_sent_spans = []
all_summary_sents = list(atomic_fact_map.keys())
picked_sents = sample(all_summary_sents, num_single_err_samples)
for picked_sent in picked_sents:
    picked_sent_idx = texts.index(picked_sent)
    for k , v in atomic_fact_map[picked_sent].items():
        corrupted_pick_sent_span = sample(v, 1)[0]
        corrupted_summary = texts[:picked_sent_idx] + [corrupted_pick_sent_span[0]] + texts[picked_sent_idx + 1:]
        corrupted_summ_sent_spans.append(('. '.join(corrupted_summary), 
                                              [(corrupted_pick_sent_span[0], 
                                              corrupted_pick_sent_span[1])]))

In [188]:
corrupted_summ_sent_spans[0]

("Greg needs to stay after hours at work. Betsy expresses frustration at the repeated situation. Johnny needs to be picked up from kindergarten. Greg asks Betsy if she can pick up Johnny. Betsy cannot pick up Johnny because she also needs to work long hours. Tuesdays are designated as Greg's days to handle kindergarten pick-up. Greg agrees to find a solution for giving up Johnny",
 [('Greg agrees to find a solution for giving up Johnny', 'giving')])

In [164]:
atomic_fact_map[picked_sent]

{'predicate': [('Their child Johnny needs to be dropped off at kindergarten',
   'dropped off at'),
  ('Their child Johnny needs to be moved up from kindergarten', 'moved'),
  ('Their child Johnny needs to not be picked up from kindergarten',
   'not picked')],
 'coreference': [("Somebody else's child Johnny needs to be picked up from kindergarten",
   "Somebody else's"),
  ("An unrelated family's child Johnny needs to be picked up from kindergarten",
   "An unrelated family's")]}

In [189]:
from random import sample

def get_single_err_samples(num_single_err_samples, atomic_fact_map, facts_list):
    corrupted_summ_sent_spans = []
    all_summary_sents = list(atomic_fact_map.keys())
    picked_sents = sample(all_summary_sents, num_single_err_samples)
    for picked_sent in picked_sents:
        picked_sent_idx = facts_list.index(picked_sent)
        for k , v in atomic_fact_map[picked_sent].items():
            corrupted_pick_sent_span = sample(v, 1)[0]
            corrupted_summary = texts[:picked_sent_idx] + [corrupted_pick_sent_span[0]] + texts[picked_sent_idx + 1:]
            corrupted_summ_sent_spans.append(('. '.join(corrupted_summary), 
                                              [(corrupted_pick_sent_span[0], 
                                              corrupted_pick_sent_span[1])]))
    return corrupted_summ_sent_spans

In [190]:
get_single_err_samples(4, atomic_fact_map, texts)

[("Greg needs to stay after hours at work. Betsy expresses frustration at the repeated situation. Johnny needs to be picked up from kindergarten. Greg asks Betsy if she can pick up Johnny. Betsy cannot pick up her sister because she also needs to work long hours. Tuesdays are designated as Greg's days to handle kindergarten pick-up. Greg agrees to find a solution for picking up Johnny",
  [('Betsy cannot pick up her sister because she also needs to work long hours',
    'cannot her sister')]),
 ("Greg needs to stay after hours at work. Betsy expresses frustration at the repeated situation. Johnny needs to be moved up from kindergarten to grade. Greg asks Betsy if she can pick up Johnny. Betsy cannot pick up Johnny because she also needs to work long hours. Tuesdays are designated as Greg's days to handle kindergarten pick-up. Greg agrees to find a solution for picking up Johnny",
  [('Johnny needs to be moved up from kindergarten to grade', 'moved grade')]),
 ("Greg needs to stay after